In [ ]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = "/home/philiprobertovich/.creds/team-week3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)


# **** SETUP ****

DATA_DIR = "./data/"
DEFAULT_BOSTON_CRIME_FILE = os.path.join(DATA_DIR, "boston_crime_data.csv")
DEFAULT_PORTLAND_CRIME_FILE = os.path.join(DATA_DIR, "portland_crime_data.csv")
PROJECT_NAME = "team-week3"
GTA_DATASET_NAME = "global_terror_attacks"



# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'boston_crime': {
        'table_name': 'boston_crime',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('incident_number', 'string', mode='REQUIRED'),
            bigquery.SchemaField('offense_code', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('offense_code_group', 'string', mode='REQUIRED'),
            bigquery.SchemaField('offense_description', 'string', mode='NULLABLE'),
            bigquery.SchemaField('district', 'string', mode='REQUIRED'),
            bigquery.SchemaField('reporting_area', 'int64',mode='NULLABLE'),
            bigquery.SchemaField('shooting', 'string',mode='NULLABLE'),
            bigquery.SchemaField('occurred_on_date','datetime64', mode='REQUIRED'),
            bigquery.SchemaField('year', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('month', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('day_of_week', 'string', mode='REQUIRED'),
            bigquery.SchemaField('hour', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('ucr_part','string', mode='NULLABLE'),
            bigquery.SchemaField('street', 'string', mode='REQUIRED'),
            bigquery.SchemaField('lat','float64', mode='REQUIRED'),
            bigquery.SchemaField('long', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('location', 'object', mode='REQUIRED')
        ]
    }
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level

logger.debug(f"Creating bigquery client")

logger.info(f"Setup Completed")